In [1]:
from pynq import Overlay
from pynq import allocate
from pprint import pprint
import numpy as np

In [30]:
# ol = Overlay('shift_left_full.bit')
# ol = Overlay('shift_right_full.bit')
ol = Overlay('grey_full.bit')
count_ip = ol.axi_gpio_0
dfx_controller = ol.dfx_controller_0

dfx_stat_ctrl_addr = 0x0
dfx_sw_trig_addr = 0x4
dfx_trig0_addr = 0x20
dfx_trig1_addr = 0x24

In [5]:
count = 5000000
count_ip.write(0x0,count)
print("Setting count to",count,"/",hex(count))

Setting count to 5000000 / 0x4c4b40


In [6]:
filename = 'shift_left_partial.bin'
file = np.fromfile(filename, dtype=np.uint8, count=-1, sep='',)

print(filename,"was loaded from SD with size of",file.size,"bytes.")

shift_left_partial.bin was loaded from SD with size of 146536 bytes.


In [7]:
left_buffer = allocate(shape=(file.size,),dtype=np.uint8, cacheable=1)
left_buffer[:] = file[:]

# partial_bit_addr = int(left_buffer.device_address/8)
partial_bit_addr = left_buffer.physical_address

print(filename,"was allocated into memory starting at address",hex(partial_bit_addr),"with block size",left_buffer.size,"bytes.")

shift_left_partial.bin was allocated into memory starting at address 0x16880000 with block size 146536 bytes.


In [9]:
print("DFX controller STATUS register:\n")
print(bin(dfx_controller.read(dfx_stat_ctrl_addr)))

DFX controller STATUS register:

0b111


In [17]:
dfx_controller.write(dfx_stat_ctrl_addr,0)

print("Putting DFX controller in shutdown state.")

Putting DFX controller in shutdown state.


In [18]:
print("DFX controller STATUS register:\n")
print(bin(dfx_controller.read(dfx_stat_ctrl_addr)))

DFX controller STATUS register:

0b10000001


In [19]:
dfx_controller.write(0x60,0)
dfx_controller.write(0x64,int(partial_bit_addr))
dfx_controller.write(0x68,int(left_buffer.size))
dfx_controller.write(0x70,1)
dfx_controller.write(0x74,int(partial_bit_addr))
dfx_controller.write(0x78,int(left_buffer.size))

print("Partial bitstream address written to dfx controller address 0x64.")
print("Partial bitstream size was written to dfx controller address 0x68.")
print("Partial bitstream address written to dfx controller address 0x74.")
print("Partial bitstream size was written to dfx controller address 0x78.")

Partial bitstream address written to dfx controller address 0x64.
Partial bitstream size was written to dfx controller address 0x68.
Partial bitstream address written to dfx controller address 0x74.
Partial bitstream size was written to dfx controller address 0x78.


In [24]:
dfx_controller.read(0x64)

0

In [25]:
print("DFX controller STATUS register:\n")
print(bin(dfx_controller.read(dfx_stat_ctrl_addr)))

DFX controller STATUS register:

0b111


In [22]:
# dfx_controller.write(dfx_stat_ctrl_addr,258)
# print("Restart DFX controller with status, mark virtual socket as full.")

dfx_controller.write(dfx_stat_ctrl_addr,2)
print("Restart DFX controller with status, mark virtual socket as empty.")

Restart DFX controller with status, mark virtual socket as empty.


In [26]:
print("DFX controller STATUS register:\n")
print(bin(dfx_controller.read(dfx_stat_ctrl_addr)))

DFX controller STATUS register:

0b111


In [29]:
dfx_controller.write(dfx_sw_trig_addr,1)

print("Software trigger 0 for partial reconfiguration set.")

Software trigger 0 for partial reconfiguration set.


In [28]:
print("DFX controller STATUS register:\n")
print(bin(dfx_controller.read(dfx_stat_ctrl_addr)))

DFX controller STATUS register:

0b100000111


In [ ]:
dfx_controller.write(dfx_stat_ctrl_addr,3)
print("Telling DFX controller to PROCEED with PR on RM.")

In [ ]:
print("DFX controller STATUS register:\n")
print(bin(dfx_controller.read(dfx_stat_ctrl_addr)))